**Первый этап:** загрузка и предобработка данных.

Вместо ручной загрузки автоматизируем процесс для удобства. Правда, придётся пожертвовать приватностью репозитория на GitHub, но ничего страшного — мы открыты миру!

In [ ]:
!pip install wget

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://github.com/Sp-V1/Scientific-research-seminar-control-task/raw/main/TEH_World_Happiness_2015_2019_Imputed.csv'

filename = wget.download(url)

dataframe = pd.read_csv('TEH_World_Happiness_2015_2019_Imputed.csv')

dataframe

,Country,Happiness rank,Happiness Score,GDP per capita,Social support,Healthy life,Freedom,Generosity,Corruption,Year
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,2015
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,2015
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,2015
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,2015
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,2015
...,...,...,...,...,...,...,...,...,...,...
777,Rwanda,152,3.334,0.35900,0.71100,0.61400,0.55500,0.21700,0.41100,2019
778,Tanzania,153,3.231,0.47600,0.88500,0.49900,0.41700,0.27600,0.14700,2019
779,Afghanistan,154,3.203,0.35000,0.51700,0.36100,NaN,0.15800,0.02500,2019
780,Central African Republic,155,3.083,0.02600,NaN,0.10500,0.22500,0.23500,0.03500,2019


Данные загружены — переходим к предобработке.

In [85]:
print("Общее количество строк данных:", len(dataframe), '\n')

print("Количество строк данных с неполной информацией:", len(dataframe.dropna()), '\n')

print("Все ли значения неотрицательны:", (dataframe.drop(columns=['Country']).fillna(0) >= 0).all().all())

Общее количество строк данных: 782 

Количество строк данных с неполной информацией: 751 

Все ли значения неотрицательны: True


Выходит, что наши данные достаточно чистые — лишь $\approx 3.96\%$ от имеющихся данных содержат пробелы — можем позволить себе исключить их из исследования.

In [86]:
upd_dataframe = dataframe.dropna()
#del dataframe # Высвобождаем память

upd_dataframe

,Country,Happiness rank,Happiness Score,GDP per capita,Social support,Healthy life,Freedom,Generosity,Corruption,Year
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,2015
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,2015
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,2015
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,2015
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,2015
...,...,...,...,...,...,...,...,...,...,...
775,Malawi,150,3.410,0.19100,0.56000,0.49500,0.44300,0.21800,0.08900,2019
776,Yemen,151,3.380,0.28700,1.16300,0.46300,0.14300,0.10800,0.07700,2019
777,Rwanda,152,3.334,0.35900,0.71100,0.61400,0.55500,0.21700,0.41100,2019
778,Tanzania,153,3.231,0.47600,0.88500,0.49900,0.41700,0.27600,0.14700,2019


**Второй этап:** разведочный анализ данных (EDA)

In [ ]:
print(list(dataframe))

#dataset.drop(columns=['Happiness rank', 'Happiness Score']).sum(axis=1)

#dataframe_copy['row_sum'] = dataframe_copy.drop(columns=['Happiness rank', 'Happiness Score', 'Year']).sum(axis=1)